In [1]:
!git clone https://github.com/iMoonLab/yolov13.git

Cloning into 'yolov13'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 527 (delta 135), reused 116 (delta 116), pack-reused 229 (from 2)
Receiving objects: 100% (527/527), 29.91 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [1]:
%cd yolov13/


/workspace/yolov13


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
!pip install thop

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO

# Load your YOLOv13 model
model = YOLO('150best.pt')

# Export with dynamic batch size
model.export(
    format='onnx',
    dynamic=True,  # Enables dynamic axes
    simplify=True,  # Simplifies the model
)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ultralytics 8.3.63 🚀 Python-3.10.12 torch-2.5.0a0+e000cf0ad9.nv24.10 CPU (unknown)
YOLOv13n summary: 535 layers, 2,448,675 parameters, 0 gradients, 6.2 GFLOPs

PyTorch: starting from '150best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.2 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: slimming with onnxslim 0.1.78...
ONNX: simplifier failure: 'str' object cannot be interpreted as an integer
ONNX: export success ✅ 2.1s, saved as '150best.onnx' (9.8 MB)

Export complete (2.7s)
Results saved to /workspace/yolov13
Predict:         yolo predict task=detect model=150best.onnx imgsz=640  
Validate:        yolo val task=detect model=150best.onnx imgsz=640 data=/workspace/yolov13/Dataset/data.yaml  
Visualize:       https://netron.app


'150best.onnx'

In [4]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession('150best.onnx')
input_name = session.get_inputs()[0].name

# Test with batch size 1
batch1 = np.random.randn(1, 3, 640, 640).astype(np.float32)
output1 = session.run(None, {input_name: batch1})

# Test with batch size 4
batch4 = np.random.randn(4, 3, 640, 640).astype(np.float32)
output4 = session.run(None, {input_name: batch4})

In [6]:
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession('150best.onnx')

# Check input/output info
print("Inputs:")
for inp in session.get_inputs():
    print(f"  {inp.name}: {inp.shape}")

print("\nOutputs:")
for out in session.get_outputs():
    print(f"  {out.name}: {out.shape}")

# Test actual output
batch1 = np.random.randn(1, 3, 640, 640).astype(np.float32)
output1 = session.run(None, {session.get_inputs()[0].name: batch1})
print(f"\nActual output shape: {output1[0].shape}")

Inputs:
  images: ['batch', 3, 'height', 'width']

Outputs:
  output0: ['batch', 'Concatoutput0_dim_1', 'anchors']

Actual output shape: (1, 8, 8400)
